***

# __<font color=red>Coronavirus disease (COVID-19) </font>__ is an infectious disease caused by a newly discovered coronavirus.

<img src="https://www.paho.org/sites/default/files/styles/flexslider_full/public/2020-03/blue-covid-banner.jpg?h=96546727&itok=xDLAoRJF" />

### Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.

### The best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face. 

### The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).


#### - [<font color=blue>Protect yourself: advice for the public</font>](https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public)

#### - [<font color=blue>Myth busters</font>](https://www.who.int/news-room/q-a-detail/coronavirus-disease-covid-19)

#### - [<font color=blue>Questions and answers</font>](https://www.who.int/news-room/q-a-detail/coronavirus-disease-covid-19)

#### - [<font color=blue>Situation reports</font>](https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports)

####     - [<font color=blue>All information on the COVID-19 outbreak</font>](https://www.who.int/emergencies/diseases/novel-coronavirus-2019)

***

In [119]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from ipywidgets import interact
import ipywidgets as widgets
import folium
from termcolor import cprint
import plotly.io as pio
from IPython.core.display import display, HTML

In [2]:

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

country_df.columns = country_df.columns.str.lower()
confirmed_df.columns = confirmed_df.columns.str.lower()
death_df.columns = death_df.columns.str.lower()
recovered_df.columns = recovered_df.columns.str.lower()

compute_confirmed = pd.melt(confirmed_df,id_vars=list(confirmed_df.iloc[:,:4].columns),var_name='Date',value_name='confirmed')
compute_deaths = pd.melt(death_df,list(death_df.iloc[:,:4].columns),var_name='Date',value_name='deaths')
compute_recovered = pd.melt(recovered_df,list(recovered_df.iloc[:,:4].columns),var_name='Date',value_name='recovered')


data = pd.merge(left=compute_confirmed,right=compute_deaths,how='outer')
data = pd.merge(left=data,right=compute_recovered,how='outer')
data['Date']  = pd.to_datetime(data['Date'])
data.rename(columns={'province/state':'state','country/region':'country'},inplace=True)
confirmed_df.drop([52,88],axis=0,inplace=True)

In [3]:
sorted_country_df = country_df.sort_values('confirmed',ascending=False)

In [156]:
upd = str(country_df['last_update'][0])
cases =sum(country_df['confirmed'])
death = sum(country_df['deaths'])
recovered = sum(country_df['recovered'])
display(HTML("<div style = 'background-color: #504e4e;  border-radius: 10px;  border: 6px solid red; padding: 30px '>" +
            "<span style='color: #fff; font-size:15px; vertical-align: super; white-space: pre-line; font-weight:bold'> As of: "  + f'{upd[:11]}\n'+"</span>" +
             "<span style='color: #fff; font-size:30px;font-weight:bold'> Confirmed Cases: "  + str(cases)+"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px; font-weight:bold'> Deaths: " + str(death) + "</span>"+
             "</div>")
       )

In [132]:
def highlight(x):
    y ='background-color:yellow;color:black'
    r ='background-color:red;color:black'
    g ='background-color:green;color:black'

    temp_df = pd.DataFrame('',index=x.index,columns=x.columns)
    temp_df.iloc[:,4] = y
    temp_df.iloc[:5,0] = r
    temp_df.iloc[:,5] = r 
    temp_df.iloc[:,6] = g
    return temp_df

# Using this interactive window you can adjust the bar and see the top countries based on confirmed cases count.

In [133]:
@interact
def show(rows=widgets.IntSlider(min=2, max=20, step=2, value=5)):
    return sorted_country_df.head(rows).style.apply(highlight,axis=None)

interactive(children=(IntSlider(value=5, description='rows', max=20, min=2, step=2), Output()), _dom_classes=(…

# Which are the most Affected Countries?

In [55]:
@interact
def graph(n=widgets.IntSlider(min=2, max=40, step=2, value=8)):
    colors=['lightslategray',]*n
    colors[0] = 'crimson'
    fig = px.bar(sorted_country_df.head(n),x='country_region',y='confirmed',hover_name='country_region',
    title=f'Top {n} Most Effected Countries',template='plotly_white',color='confirmed')
    fig.update_layout()
    fig.show(renderer='notebook')

interactive(children=(IntSlider(value=8, description='n', max=40, min=2, step=2), Output()), _dom_classes=('wi…

# Many people have lost their loved ones in this pandemic. Using this graph we can visualize which countries have suffered the most. The graph depicts the percentage Distribution of the Number of People who died due to Corona virus.

In [17]:
@interact
def pie(top = widgets.IntSlider(min=5,max=100,step=2)):
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    fig.add_trace(go.Pie(labels=sorted_country_df['country_region'].head(top), values=sorted_country_df['confirmed'].head(top)),1, 1)
    fig.add_trace(go.Pie(labels=sorted_country_df['country_region'].head(top), values=sorted_country_df['deaths'].head(top)),
                1, 2)

    # Use `hole` to create a donut-like pie chart
    fig.update_traces(hole=.3, hoverinfo="label+percent+name",textposition='inside', textinfo='percent+label')

    fig.update_layout(
        title_text=f'Top {top} Countries with death and confirmed percentiles',
        # Add annotations in the center of the donut pies.
        annotations=[dict(text='Confirmed', x=0.191, y=0.5, font_size=20, showarrow=False),
                    dict(text='Deaths', x=0.80, y=0.5, font_size=20, showarrow=False)])
    fig.show(renderer='notebook')

interactive(children=(IntSlider(value=5, description='top', min=5, step=2), Output()), _dom_classes=('widget-i…

# Are you looking for what is the current scenario of our home country. How well we are fighting this virus.? Enter your country's name and explore what is the current scenario of your country.

In [8]:
@interact
def give_plot(Place='India'):
    if place in list(sorted_country_df['country_region']):
        fig = make_subplots(rows=3,cols=1,subplot_titles=['Recovered','Confirmed','Deaths'],x_title='Date',y_title='No of People',vertical_spacing=0.1)
        fig.add_trace(go.Scatter(x=data[data['country']==Place]['Date'],y=data[data['country']==Place]['recovered'],name='Recovered',mode='lines+markers'),row=1,col=1)
        fig.add_trace(go.Scatter(x=data[data['country']==Place]['Date'],y=data[data['country']==Place]['confirmed'],name='Confirmed',mode='lines+markers'),row=2,col=1)
        fig.add_trace(go.Scatter(x=data[data['country']==Place]['Date'],y=data[data['country']==Place]['deaths'],name='Deaths',mode='lines+markers'),row=3,col=1)
        fig.update_layout(height=800,title_text=f'Scenario in {Place}',template='plotly_white')
        fig.show(renderer='notebook')
    else:
        print("This country's data is not collected!")

interactive(children=(Text(value='India', description='place'), Output()), _dom_classes=('widget-interact',))

# What is the mortality Rate of yout Country?
### Mortality rate, or death rate, is a measure of the number of deaths in a particular population, scaled to the size of that population, per unit of time.

In [160]:
@interact
def mortality(place='India'):
    if place in list(sorted_country_df['country_region']):
        fig = go.Figure(go.Indicator(
            mode = "gauge+number",
            value = float(sorted_country_df[sorted_country_df['country_region']==place]['mortality_rate']),
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {'text': f"Mortality Rate of {place}"}))

        fig.show(renderer='notebook')
    else:
        print("This country's data is not collected!")

interactive(children=(Text(value='India', description='place'), Output()), _dom_classes=('widget-interact',))

# How well are we fighting Corona Virus. What is the Global spread of the virus?
### Circles depicts no of cases. Bigger the circle more confirmed cases in that region

In [9]:
world = folium.Map(location =[11,0],zoom_start=2.5,max_zoom=6,min_zoom=2.5,tiles='cartodbpositron')

for i in range(len(confirmed_df)):
    folium.Circle(location=[ confirmed_df.iloc[i]['lat'],confirmed_df.iloc[i]['long']],
    radius=(int( np.log(confirmed_df.iloc[i,-1]+1.00001))+0.2)*12000,
    fill=True,
    color='red',
    fill_color='red').add_to(world)
world